<a href="https://colab.research.google.com/github/sebastianmunozvasq/CFD/blob/main/TP4_vac%C3%ADo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###  Trabajo práctico Módulo 4

## Protocolo de emergencia frente a una reacción de fusión nuclear descontrolada

#### MOOC: Transferencia de Calor y Masa Computacional

Imagine que usted trabaja en el laboratorio Nacional de Los Álamos en un proyecto que busca obtener cantidades sin precedentes de energía mediante fusión nuclear. Usted está a cargo del protocolo de seguridad experimental frente a una reacción nuclear descontrolada. En particular, se le pide que diseñe un sistema de mitigación de calor en caso de una reacción nuclear descontrolada.

La Figura 1 una esfera de 16 cm de radio que contiene 1 kg de la mezcla Deuterio-Tritio. A partir de una nueva tecnología de campos gravitacionales, los experimentalistas afirman que lograron realizar una reacción controlada de fusión a 600°C. Sin embargo, si el enfriamiento falla, la temperatura de la esfera aumentará producto de la reacción de fusión.

<div>
  <img src="/figs/fig1_DT_TP4.png" width="600"/>
  <figcaption>Figura 1: Protocolo de seguridad para esfera Deuterio-Tritio</figcaption>
</div>


Este aumento de temperatura es muy rápido y se puede modelar como un término volumétrico de generación de calor:


$$ S = 1 \text{MW}/\text{m}^3 $$

 En caso de llegar a 1100°C, la fusión se volverá descontrolada y todo el laboratorio nacional se desintegraría en segundos junto a todo su personal.

La ecuación de conservación de energía dentro de la esfera es

$$ \rho_1 \hat{c}_{p}\frac{\partial T}{\partial t} = \frac{k}{r^2} \frac{\partial}{\partial r} \left(r^2 \frac{\partial T}{\partial r} \right) + S, \hspace{2cm} 0 < r < R_1 $$

En $r=0$, el perfil de temperatura es simétrico lo que se traduce en la siguiente condición de borde:

$$ \frac{\partial T}{\partial r}|_{r=0} = 0 $$

El protocolo de seguridad actual consiste en lanzar la esfera a un receptáculo muy grande con agua con hielo. En una emergencia, esto produciría la evaporación súbita del agua adyacente a la esfera. En $r=R$, existe enfriamiento de la superficie de la esfera por evaporación de película. Esto se traduce en la siguiente condición de borde:

$$ -k \frac{\partial T}{\partial r}|_{r=R} = h(T|_{r=R} - T_\infty) $$

El sistema actual está configurado con las siguientes propiedades termofísicas:

#### Esfera de material compuesto poroso con Deuterio-Tritio:
* Conductividad térmica: $ k_1 = 10 \text{ W/mK}$
* Calor específico: $\hat{c}_p = 900 \text{ J/kg/°C}$
* La densidad se puede calcular a partir de los datos experimentales
* Condiciones iniciales: $T(t=0) = 873.15 K $

#### Transferencia de calor
* Temperatura de ebullición del agua a presión atmosférica: $T_{\infty} = 373.15 K$
* Coeficiente de transferencia de calor por cambio de fase: $h = 10000 \text{W}/\text{m}^2/\text{K}$

Se le solicita evaluar si el sistema de seguridad es suficiente para prevenir que la temperatura en cualquier punto y momento sea inferior a 1100°C.

#### Parte 1. Discretice la EDP utilizando diferencias finitas de segundo orden y el método FTCS

#### 1.1 Discretice los nodos interiores, utilizando:
* Diferencias finitas hacia adelante de primer orden en el tiempo para la derivada temporal
* Diferencias finitas centrales de segundo orden en el espacio para las derivadas espaciales

#### 1.2 Encuentre los coeficientes de la matriz A en la ecuación de evolución

#### 1.3) Discretice las condiciones de borde
* Utilice diferencias finitas hacia adelante o atrás, según corresponda, de segundo orden.
* Se le recomienda despejar las temperaturas del nodo inicial $T^{t+1}_0$ y el nodo final $T_N^{t}$ en función de los nodos adyacentes.
* Esto le permitirá implementar el método de manera más sencilla.
* En esta implementación, usted deberá dejar la primera y última fila de la matriz de coeficientes A vacía, así como también el primer y último elemento del vector $\mathbf{b}$ vacío.

### Parte 2) Implementación del método numérico

#### 2.1 - Importar módulos

In [ ]:
# Cálculos numéricos
import numpy as np

# Gráficos
import matplotlib.pyplot as plt

# Mapas de colores
from matplotlib import cm

#### 2.2 Declaración de parámetros físicos

* Defina las propiedades termofísicas del problema
* Calcule la densidad de la esfera interna
* Calcule la difusividad térmica

#### 2.3 Definición de los parámetros de grilla y computacionales

Se le solicita resolver el problema considerando los siguientes parámetros:

* 100 nodos en la dirección radial
* Un paso de tiempo que corresponda a un Número de Fourier computacional igual a 0.45,
$$\text{Fo}_c = 0.45$$

In [ ]:
R = 0.16  # Radio de la esfera en metros
V = (4/3) * np.pi * R**3  # Volumen de la esfera
m = 1  # Masa de la esfera en kg
rho = m / V  # Densidad de la esfera
k = 10  # Conductividad térmica en W/mK
c_p = 900  # Calor específico en J/kg/K
S = 1e6  # Fuente de calor en W/m^3
h = 10000  # Coeficiente de transferencia de calor en W/m^2/K
T0 = 873.15  # Temperatura inicial en K
T_inf = 373.15  # Temperatura del sumidero de calor en K
Nr = 100  # Número de nodos espaciales

T_max_allowed=1100

# Crear un dominio discretizado
r = np.linspace(0, R, Nr)  # Dominio radial desde 0 hasta R (0.16 metros)
dr = r[1] - r[0]  # Paso espacial

# Calcular la difusividad térmica
alpha = k / (rho * c_p)  # Difusividad térmica
Fo_c = 0.45  # Número de Fourier computacional
dt = Fo_c * dr**2 / alpha  # Paso temporal


# Construcción de matriz A y vector del lado derecho
A = np.zeros((Nr, Nr))
b = np.zeros(Nr)

# Condición de borde en r = 0
A[0, 0] = 1
A[0, 1] = -1
b[0] = 0

# Condición de borde en r = R
A[-1, -1] = k / dr + h
A[-1, -2] = -k / dr
b[-1] = h * T_inf



#### 2.4 Rellenar matriz A - nodos interiores

In [ ]:
# Iteración en nodos interiores
for i in range(1, Nr - 1):
    A[i, i - 1] = k / (rho * c_p * dr**2) - k / (rho * c_p * 2 * r[i] * dr)
    A[i, i] = -2 * k / (rho * c_p * dr**2)
    A[i, i + 1] = k / (rho * c_p * dr**2) + k / (rho * c_p * 2 * r[i] * dr)
    b[i] = S / (rho * c_p)


#### 2.5 - Definir parámetros FTCS

In [ ]:
# Tiempo inicial de integración
t_initial = 0  # segundos

# Tiempo final de integración: debe ser adecuado para responder las preguntas del enunciado
t_final = 5  # segundos (ajustable)

# Grabe los resultados después de una fracción que estime conveniente. Por ejemplo, t = 3 s:
record_interval = 1  # segundos para grabar resultados


#### 2.6 Implementar el algoritmo FTCS

In [ ]:
## Algoritmo FTCS

# Se sugiere que usted defina una función, pero no es estrictamente necesario

# Definición de función
def FTCS(A, b, t_initial, t_final, record_interval):
    time_results = []
    t_current = t_initial
    T = np.ones(Nr) * T0
    T_copy = np.copy(T)
    T_max_history = T0
    temperature_results = []

    while t_current < t_final:
        T_new = T + dt * (np.dot(A, T) + b)
        t_current += dt
        T_new[0] = T_new[1]  # Condición de borde en r = 0
        T_new[-1] = (k * T_new[-2] + h * dr * T_inf) / (k + h * dr)  # Condición de borde en r = R
        T_max_current = np.max(T_new)

        # Verificar si la temperatura máxima excede el límite permitido
        if T_max_current >= T_max_allowed:
            print(f"Temperatura máxima excedida: {T_max_current} K. Terminando la simulación.")
            break

        if T_max_current > T_max_history:
            T_max_history = T_max_current
            time_results.append(t_current)
            temperature_results.append(np.copy(T_new))

        T = np.copy(T_new)

        if t_current % record_interval < dt:
            time_results.append(t_current)
            temperature_results.append(np.copy(T))

    return time_results, temperature_results, T_max_history

### Preguntas quiz

#### P1) Calcule la temperatura en K en el centro de la esfera después de 5 segundos de haber iniciado el protocolo de seguridad

In [ ]:
# Resuelva la iteración para 5 segundos
t_final=5
(time_results, temperature_results, T_max_history) = FTCS(A, b, t_initial, t_final, record_interval)
# Imprima la temperatura en el centro de la esfera
# Temperatura en el centro de la esfera
print(f"Temperatura en el centro de la esfera a los 5 segundos: {temperature_results[-1][0]} K")

Temperatura en el centro de la esfera a los 5 segundos: 967.372110123629 K


#### P2) Encuentre el flujo de calor en W removido de la esfera por el sumidero de calor después de 24 segundos de haber iniciado el protocolo de seguridad

Calculamos el flujo de calor que sale de la esfera a los 24 segundos:

$$\dot{Q} = 4 \pi R^2 h (T|_{N_R}(t=24 s)-T_{\infty}) $$

In [ ]:
# Resuelva para 24 segundos
t_final=24
(time_results, temperature_results, T_max_history) = FTCS(A, b, t_initial, t_final, record_interval)
# Extraiga la temperatura de la pared
print(f"Temperatura de la pared a los 24 segundos: {temperature_results[-1][-1]} K")

# Calcule e imprima el flujo de calor después de 24 segundos
print(f"Flujo de calor removido de la esfera a los 24 segundos: {4 * np.pi * R**2 * h * (temperature_results[-1][-1] - T_inf)} W")


Temperatura de la pared a los 24 segundos: 381.368027581205 K
Flujo de calor removido de la esfera a los 24 segundos: 26437.319757938723 W


#### P3 y P4) Encuentre la temperatura máxima de la esfera y el tiempo en que esta temperatura se alcanza

Sugerencias:

* Para sus cálculos, considere un tiempo de simulación lo suficientemente largo para asegurarse de que el perfil de temperatura llega a un estado estacionario.
* Modifique la iteración FTCS para almacenar la temperatura máxima en un tiempo determinado y el tiempo en la que esta ocurra.

In [ ]:
# Resolvemos para un tiempo suficientemente largo a partir de los perfiles de temperatura
t_final=200
def FTCS_v2(A, b, t_initial, t_final, record_interval):
    # Lista en que se acumulan los tiempos donde se graban los perfiles de temperatura
    time_results = []
    t_current = t_initial

    # Inicializar vector que contiene la temperatura inicial de la esfera
    T = np.ones(Nr) * T0

    # Copie la temperatura T en otra variable, utilizando la función np.copy()
    T_copy = np.copy(T)

    # Inicializar temperatura máxima
    T_max_history = T0
    T_max_time = t_initial

    # Lista con los perfiles de temperatura para cada write_interval
    temperature_results = []

    # Iteración de evolución
    while t_current < t_final:
        # Actualizar nodos interiores
        T_new = T + dt * (np.dot(A, T) + b)

        # Actualizar tiempo
        t_current += dt

        # Actualizar condiciones de borde
        T_new[0] = T_new[1]  # Aplicar condición de borde en r = 0 (simetría radial)
        T_new[-1] = (k * T_new[-2] + h * dr * T_inf) / (k + h * dr)  # Aplicar condición de borde en r = R

        # Calcular la temperatura máxima en el tiempo actual
        T_max_current = np.max(T_new)

        # Guardar la temperatura y tiempo si se excede la temperatura máxima histórica
        if T_max_current > T_max_history:
            T_max_history = T_max_current
            T_max_time = t_current
            time_results.append(t_current)
            temperature_results.append(np.copy(T_new))

        # Crear una copia de la nueva temperatura
        T = np.copy(T_new)

        # Definir un control de flujo para guardar el perfil de temperatura
        # cada vez que se cumpla un intervalo de tiempo previamente definido
        if t_current % record_interval < dt:
            # Guardamos el perfil de temperatura
            time_results.append(t_current)
            # Guardamos el tiempo
            temperature_results.append(np.copy(T))

    # Parámetros a retornar (en caso de definir una función)
    return time_results, temperature_results, T_max_history, T_max_time

(time_results, temperature_results, T_max_history, T_max_time) = FTCS_v2(A, b, t_initial, t_final, record_interval)
print(f"Temperatura máxima de la esfera: {T_max_history} K")
print(f"Tiempo en que se alcanza la temperatura máxima: {T_max_time} s")

Temperatura máxima de la esfera: 1051.7685078619606 K
Tiempo en que se alcanza la temperatura máxima: 18.305695740179814 s


### P5) Calcule la temperatura promedio después de 3600 s

Sugerencia: recuerde que el promedio de una función escalar en coordenadas cilíndricas es:

$$ \left\langle T\right\rangle=\frac{\int_{0}^{2\pi}\int_{0}^{\pi}\int_{0}^{R}{T\left(r\right)r^2drd\theta d\varphi}}{\int_{0}^{2\pi}\int_{0}^{\pi}\int_{0}^{R}{r^2drd\theta d\varphi}} $$

In [ ]:
# Resuelva para 60 segundos
t_final=60
(time_results, temperature_results, T_max_history) = FTCS(A, b, t_initial, t_final, record_interval)

# Calcule la temperatura promedio después de una hora utilizando el método trapezoidal o similar

from scipy.integrate import trapezoid

# Definimos la función que devuelve la temperatura promedio
def temperatura_promedio_esfera(T, r):
    # Calculamos el numerador usando la regla del trapezoide
    numerador = trapezoid(T * r**2, r)

    # Calculamos el denominador analíticamente
    denominador = R**3 / 3

    # Calculamos la temperatura promedio
    T_promedio = numerador / denominador

    return T_promedio
T_promedio = temperatura_promedio_esfera(temperature_results[-1], r)
print(f"Temperatura promedio de la esfera después de 60 s: {T_promedio} K")

Temperatura promedio de la esfera después de 60 s: 639.2649422312178 K
